# The notebook contains
### Code for _Bulyan_ aggregation algorithm, *when gradient updates of benign clients are unknown to adversary*
### Evaluation of all the attacks (Fang, LIE, and our AGR-agnstic) on Multi-krum, except our AGR-tailored attack on Bulyan

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from __future__ import print_function
import argparse, os, sys, csv, shutil, time, random, operator, pickle, ast, math, json
import numpy as np
import pandas as pd
from torch.optim import Optimizer
import torch.nn.functional as F
import torch
import pickle
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torch.multiprocessing as mp

sys.path.insert(0,'./../utils/')
from logger import *
from eval import *
from misc import *

from femnist_normal_train import *
from femnist_util import *
from adam import Adam
from sgd import SGD
import torchvision.transforms as transforms
import torchvision.datasets as datasets

## Get the FEMNIST dataset; we use [LEAF framework](https://leaf.cmu.edu/)

In [3]:
user_tr_data = []
user_tr_labels = []

for i in range(34):
    f = '/mnt/nfs/work1/amir/vshejwalkar/leaf/data/femnist/data/train/all_data_%d_niid_0_keep_0_train_9.json'%i
    with open(f, 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    
    for user in obj['users']:
        user_tr_data.append(obj['user_data'][user]['x'])
        user_tr_labels.append(obj['user_data'][user]['y'])

user_te_data = []
user_te_labels = []

for i in range(34):
    f = '/mnt/nfs/work1/amir/vshejwalkar/leaf/data/femnist/data/test/all_data_%d_niid_0_keep_0_test_9.json'%i
    with open(f, 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    
    for user in obj['users']:
        user_te_data.append(obj['user_data'][user]['x'])
        user_te_labels.append(obj['user_data'][user]['y'])

In [4]:
user_tr_data_tensors=[]
user_tr_label_tensors=[]

for i in range(len(user_tr_data)):
    
    user_tr_data_tensor=torch.from_numpy(np.array(user_tr_data[i])).type(torch.FloatTensor)
    user_tr_label_tensor=torch.from_numpy(np.array(user_tr_labels[i])).type(torch.LongTensor)

    user_tr_data_tensors.append(user_tr_data_tensor)
    user_tr_label_tensors.append(user_tr_label_tensor)
    
    # print('user %d tr len %d'%(i,len(user_tr_data_tensor)))
print("number of clients: ", len(user_tr_data_tensors))

number of clients:  3400


In [5]:
te_data = np.concatenate(user_te_data, 0)
te_labels = np.concatenate(user_te_labels)
te_len = len(te_labels)

te_data_tensor = torch.from_numpy(te_data[:(te_len//2)]).type(torch.FloatTensor)
te_label_tensor = torch.from_numpy(te_labels[:(te_len//2)]).type(torch.LongTensor)

val_data_tensor = torch.from_numpy(te_data[(te_len//2):]).type(torch.FloatTensor)
val_label_tensor = torch.from_numpy(te_labels[(te_len//2):]).type(torch.LongTensor)

## Code for Bulyan aggregation algorithm

In [6]:
def bulyan(all_updates, n_attackers):
    nusers = all_updates.shape[0]
    bulyan_cluster = []
    candidate_indices = []
    remaining_updates = all_updates
    all_indices = np.arange(len(all_updates))

    while len(bulyan_cluster) < (nusers - 2 * n_attackers):
        distances = []
        for update in remaining_updates:
            distance = torch.norm((remaining_updates - update), dim=1) ** 2
            distances = distance[None, :] if not len(distances) else torch.cat((distances, distance[None, :]), 0)

        distances = torch.sort(distances, dim=1)[0]

        scores = torch.sum(distances[:, :len(remaining_updates) - 2 - n_attackers], dim=1)
        indices = torch.argsort(scores)[:len(remaining_updates) - 2 - n_attackers]

        candidate_indices.append(all_indices[indices[0].cpu().numpy()])
        all_indices = np.delete(all_indices, indices[0].cpu().numpy())
        bulyan_cluster = remaining_updates[indices[0]][None, :] if not len(bulyan_cluster) else torch.cat((bulyan_cluster, remaining_updates[indices[0]][None, :]), 0)
        remaining_updates = torch.cat((remaining_updates[:indices[0]], remaining_updates[indices[0] + 1:]), 0)

    # print('dim of bulyan cluster ', bulyan_cluster.shape)

    n, d = bulyan_cluster.shape
    param_med = torch.median(bulyan_cluster, dim=0)[0]
    sort_idx = torch.argsort(torch.abs(bulyan_cluster - param_med), dim=0)
    sorted_params = bulyan_cluster[sort_idx, torch.arange(d)[None, :]]

    return torch.mean(sorted_params[:n - 2 * n_attackers], dim=0), np.array(candidate_indices)

## Code for Multi-krum aggregation algorithm

In [7]:
def multi_krum(all_updates, n_attackers, multi_k=False):
    nusers = all_updates.shape[0]
    candidates = []
    candidate_indices = []
    remaining_updates = all_updates
    all_indices = np.arange(len(all_updates))

    while len(remaining_updates) > 2 * n_attackers + 2:
        distances = []
        for update in remaining_updates:
            distance = torch.norm((remaining_updates - update), dim=1) ** 2
            distances = distance[None, :] if not len(distances) else torch.cat((distances, distance[None, :]), 0)

        distances = torch.sort(distances, dim=1)[0]
        scores = torch.sum(distances[:, :len(remaining_updates) - 2 - n_attackers], dim=1)
        indices = torch.argsort(scores)[:len(remaining_updates) - 2 - n_attackers]

        candidate_indices.append(all_indices[indices[0].cpu().numpy()])
        all_indices = np.delete(all_indices, indices[0].cpu().numpy())
        candidates = remaining_updates[indices[0]][None, :] if not len(candidates) else torch.cat((candidates, remaining_updates[indices[0]][None, :]), 0)
        remaining_updates = torch.cat((remaining_updates[:indices[0]], remaining_updates[indices[0] + 1:]), 0)
        if not multi_k:
            break
    # print(len(remaining_updates))
    aggregate = torch.mean(candidates, dim=0)
    return aggregate, np.array(candidate_indices)


## Code for Fang attack on Bulyan
### Fang attacks on MKrum and Bulyan are the same

In [8]:
def compute_lambda(all_updates, model_re, n_attackers):

    distances = []
    n_benign, d = all_updates.shape
    for update in all_updates:
        distance = torch.norm((all_updates - update), dim=1)
        distances = distance[None, :] if not len(distances) else torch.cat((distances, distance[None, :]), 0)

    distances[distances == 0] = 10000
    distances = torch.sort(distances, dim=1)[0]
    scores = torch.sum(distances[:, :n_benign - 2 - n_attackers], dim=1)
    min_score = torch.min(scores)
    term_1 = min_score / ((n_benign - n_attackers - 1) * torch.sqrt(torch.Tensor([d]))[0])
    max_wre_dist = torch.max(torch.norm((all_updates - model_re), dim=1)) / (torch.sqrt(torch.Tensor([d]))[0])

    return (term_1 + max_wre_dist)


def get_malicious_updates_fang(all_updates, model_re, deviation, n_attackers):

    lamda = compute_lambda(all_updates, model_re, n_attackers)

    threshold = 1e-5
    mal_update = []

    while lamda > threshold:
        mal_update = (-lamda * deviation)
        mal_updates = torch.stack([mal_update] * n_attackers)
        mal_updates = torch.cat((mal_updates, all_updates), 0)

        # print(mal_updates.shape, n_attackers)
        agg_grads, krum_candidate = multi_krum(mal_updates, n_attackers, multi_k=False)
        if krum_candidate < n_attackers:
            # print('successful lamda is ', lamda)
            return mal_update
        else:
            mal_update = []

        lamda *= 0.5

    if not len(mal_update):
        mal_update = (model_re - lamda * deviation)
        
    return mal_update

In [12]:
resume=0
nepochs=1500
gamma=.1
fed_lr=0.001

criterion = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()
batch_size = 100
schedule = [2000]

aggregation = 'bulyan'
at_type = 'fang'
chkpt = './' + aggregation
epoch_num = 0

at_fractions = [20]

for at_fraction in at_fractions:

    fed_model = mnist_conv().cuda()
    fed_model.apply(weights_init)
    optimizer_fed = Adam(fed_model.parameters(), lr=fed_lr)

    print('==> Initializing global model')
    epoch_num = 0
    n_attacker = 0
    best_global_acc=0
    best_global_te_acc=0

    while epoch_num <= nepochs:
        user_grads = []

        # The following condition is necessary for Krum/Multi-krum/Bulyan to work
        while n_attacker < 4:
            round_users = np.random.choice(3400, 60)
            n_attacker = np.sum(round_users < (34*at_fraction))

        if n_attacker > 14:
            print ('n_attackers actual %d adjusted 14' % n_attacker)
            n_attacker = 14

        at_idx = []
        attacker_count = 0
        for i in round_users:
            if i < (34*at_fraction) and attacker_count < n_attacker:
                at_idx.append(i)
                attacker_count += 1
                continue

            inputs = user_tr_data_tensors[i]
            targets = user_tr_label_tensors[i]

            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

            outputs = fed_model(inputs)
            loss = criterion(outputs, targets)
            optimizer_fed.zero_grad()
            loss.backward(retain_graph=True)

            param_grad=[]
            for param in fed_model.parameters():
                param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

            user_grads=param_grad[None,:] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]),0)    

        if n_attacker > 0:
            attacker_grads = []
            n_attacker_ = max(1, n_attacker**2//60)
            for i in at_idx:

                inputs = user_tr_data_tensors[i]
                targets = user_tr_label_tensors[i]

                inputs, targets = inputs.cuda(), targets.cuda()
                inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

                outputs = fed_model(inputs)
                loss = criterion(outputs, targets)
                optimizer_fed.zero_grad()
                loss.backward(retain_graph=True)

                param_grad=[]
                for param in fed_model.parameters():
                    param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

                attacker_grads=param_grad[None,:] if len(attacker_grads)==0 else torch.cat((attacker_grads,param_grad[None,:]),0)

            mal_updates = []
            if at_type == 'fang':
                agg_grads = torch.mean(attacker_grads, 0)
                deviation = torch.sign(agg_grads)
                mal_update = get_malicious_updates_fang(attacker_grads, agg_grads, deviation, n_attacker_)

            elif at_type == 'our-agr':
                agg_grads = torch.mean(attacker_grads, 0)
                mal_update = our_attack_mkrum(attacker_grads, agg_grads, n_attacker_, dev_type='sign')
            
        if not len(mal_updates):
            mal_updates = torch.stack([mal_update] * n_attacker)
        malicious_grads = torch.cat((mal_updates, user_grads), 0)    
        
        if malicious_grads.shape[0] != 60: 
            print('malicious grads shape ', malicious_grads.shape)
            sys.exit()

        agg_grads, krum_candidate = bulyan(malicious_grads, n_attacker)
            
        start_idx=0

        if epoch_num in schedule:
            for param_group in optimizer_fed.param_groups:
                param_group['lr'] *= gamma
                print('New learnin rate ', param_group['lr'])

        optimizer_fed.zero_grad()

        model_grads=[]

        for i, param in enumerate(fed_model.parameters()):
            param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
            start_idx=start_idx+len(param.data.view(-1))
            param_=param_.cuda()
            model_grads.append(param_)

        optimizer_fed.step(model_grads)

        val_loss, val_acc = test(val_data_tensor,val_label_tensor,fed_model,criterion,use_cuda)
        te_loss, te_acc = test(te_data_tensor,te_label_tensor, fed_model, criterion, use_cuda)

        is_best = best_global_acc < val_acc

        best_global_acc = max(best_global_acc, val_acc)

        if is_best:
            best_global_te_acc = te_acc

        if epoch_num % 20 == 0:
            print('%s: at %s at_frac %.1f n_at %d n_mal_sel %d e %d fed_model val loss %.4f val acc %.4f best val_acc %f te_acc %f'%(aggregation, at_type, at_fraction, n_attacker, np.sum(krum_candidate < n_attacker), epoch_num, val_loss, val_acc, best_global_acc,best_global_te_acc))

        epoch_num+=1

==> Initializing global model
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 0 fed_model val loss 4.0347 val acc 5.1457 best val_acc 5.145696 te_acc 5.397961
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 9 e 20 fed_model val loss 3.8071 val acc 16.5852 best val_acc 19.836285 te_acc 22.371808
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 40 fed_model val loss 3.4244 val acc 27.2086 best val_acc 28.467360 te_acc 31.576915
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 60 fed_model val loss 3.0778 val acc 27.1031 best val_acc 30.905066 te_acc 34.480540
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 80 fed_model val loss 2.7368 val acc 33.3170 best val_acc 33.317030 te_acc 36.771520
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 100 fed_model val loss 2.6942 val acc 34.3029 best val_acc 35.124073 te_acc 38.992998
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 120 fed_model val loss 2.4517 val acc 38.1976 best val_acc 38.753604 te_acc 42.787273
bulyan: at fan

bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 1200 fed_model val loss 1.4577 val acc 62.6442 best val_acc 65.457681 te_acc 66.469316
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 1220 fed_model val loss 1.5330 val acc 60.8912 best val_acc 65.457681 te_acc 66.469316
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 1240 fed_model val loss 1.6787 val acc 58.3093 best val_acc 65.457681 te_acc 66.469316
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 1260 fed_model val loss 1.7443 val acc 57.4727 best val_acc 65.457681 te_acc 66.469316
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 1280 fed_model val loss 1.8586 val acc 55.6914 best val_acc 65.457681 te_acc 66.469316
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 1300 fed_model val loss 2.0829 val acc 52.0619 best val_acc 65.457681 te_acc 66.469316
bulyan: at fang at_frac 20.0 n_at 13 n_mal_sel 13 e 1320 fed_model val loss 2.0442 val acc 52.8393 best val_acc 65.457681 te_acc 66.469316
bulyan: at fang at_frac 20.

## Our first AGR-agnostic attack - Min-Max

In [13]:
def our_attack_dist(all_updates, model_re, n_attackers, dev_type='unit_vec'):

    if dev_type == 'unit_vec':
        deviation = model_re / torch.norm(model_re)  # unit vector, dir opp to good dir
    elif dev_type == 'sign':
        deviation = torch.sign(model_re)
    elif dev_type == 'std':
        deviation = torch.std(all_updates, 0)

    lamda = torch.Tensor([50.0]).float().cuda()
    # print(lamda)
    threshold_diff = 1e-5
    lamda_fail = lamda
    lamda_succ = 0
    
    distances = []
    for update in all_updates:
        distance = torch.norm((all_updates - update), dim=1) ** 2
        distances = distance[None, :] if not len(distances) else torch.cat((distances, distance[None, :]), 0)
    
    max_distance = torch.max(distances)
    del distances

    while torch.abs(lamda_succ - lamda) > threshold_diff:
        mal_update = (model_re - lamda * deviation)
        distance = torch.norm((all_updates - mal_update), dim=1) ** 2
        max_d = torch.max(distance)
        
        if max_d <= max_distance:
            # print('successful lamda is ', lamda)
            lamda_succ = lamda
            lamda = lamda + lamda_fail / 2
        else:
            lamda = lamda - lamda_fail / 2

        lamda_fail = lamda_fail / 2

    mal_update = (model_re - lamda_succ * deviation)
    
    return mal_update

In [14]:
resume=0
nepochs=1500
gamma=.1
fed_lr=0.001

criterion = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()
batch_size = 100
schedule = [2000]

aggregation = 'bulyan'
at_type = 'min-max'
chkpt = './' + aggregation
epoch_num = 0

at_fractions = [20]

for at_fraction in at_fractions:

    fed_model = mnist_conv().cuda()
    fed_model.apply(weights_init)
    optimizer_fed = Adam(fed_model.parameters(), lr=fed_lr)

    print('==> Initializing global model')
    epoch_num = 0
    n_attacker = 0
    best_global_acc=0
    best_global_te_acc=0

    while epoch_num <= nepochs:
        user_grads = []

        # The following condition is necessary for Krum/Multi-krum/Bulyan to work
        while n_attacker < 4:
            round_users = np.random.choice(3400, 60)
            n_attacker = np.sum(round_users < (34*at_fraction))

        if n_attacker > 14:
            print ('n_attackers actual %d adjusted 14' % n_attacker)
            n_attacker = 14

        at_idx = []
        attacker_count = 0
        for i in round_users:
            if i < (34*at_fraction) and attacker_count < n_attacker:
                at_idx.append(i)
                attacker_count += 1
                continue

            inputs = user_tr_data_tensors[i]
            targets = user_tr_label_tensors[i]

            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

            outputs = fed_model(inputs)
            loss = criterion(outputs, targets)
            optimizer_fed.zero_grad()
            loss.backward(retain_graph=True)

            param_grad=[]
            for param in fed_model.parameters():
                param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

            user_grads=param_grad[None,:] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]),0)    

        if n_attacker > 0:
            attacker_grads = []
            n_attacker_ = max(1, n_attacker**2//60)
            for i in at_idx:

                inputs = user_tr_data_tensors[i]
                targets = user_tr_label_tensors[i]

                inputs, targets = inputs.cuda(), targets.cuda()
                inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

                outputs = fed_model(inputs)
                loss = criterion(outputs, targets)
                optimizer_fed.zero_grad()
                loss.backward(retain_graph=True)

                param_grad=[]
                for param in fed_model.parameters():
                    param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

                attacker_grads=param_grad[None,:] if len(attacker_grads)==0 else torch.cat((attacker_grads,param_grad[None,:]),0)

            mal_updates = []
            if at_type == 'fang':
                agg_grads = torch.mean(attacker_grads, 0)
                deviation = torch.sign(agg_grads)
                mal_update = get_malicious_updates_fang(attacker_grads, agg_grads, deviation, n_attacker_)
            elif at_type == 'our-agr':
                agg_grads = torch.mean(attacker_grads, 0)
                mal_update = our_attack_mkrum(attacker_grads, agg_grads, n_attacker_, dev_type='sign')
            elif at_type == 'min-max':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_dist(attacker_grads, agg_grads, n_attacker_, dev_type='sign')
            elif at_type == 'min-sum':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_score(attacker_grads, agg_grads, n_attacker_, dev_type='sign')

        if not len(mal_updates):
            mal_updates = torch.stack([mal_update] * n_attacker)
        malicious_grads = torch.cat((mal_updates, user_grads), 0)    
        
        if malicious_grads.shape[0] != 60: 
            print('malicious grads shape ', malicious_grads.shape)
            sys.exit()

        agg_grads, krum_candidate = bulyan(malicious_grads, n_attacker)

        start_idx=0

        if epoch_num in schedule:
            for param_group in optimizer_fed.param_groups:
                param_group['lr'] *= gamma
                print('New learnin rate ', param_group['lr'])

        optimizer_fed.zero_grad()

        model_grads=[]

        for i, param in enumerate(fed_model.parameters()):
            param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
            start_idx=start_idx+len(param.data.view(-1))
            param_=param_.cuda()
            model_grads.append(param_)

        optimizer_fed.step(model_grads)

        val_loss, val_acc = test(val_data_tensor,val_label_tensor,fed_model,criterion,use_cuda)
        te_loss, te_acc = test(te_data_tensor,te_label_tensor, fed_model, criterion, use_cuda)

        is_best = best_global_acc < val_acc

        best_global_acc = max(best_global_acc, val_acc)

        if is_best:
            best_global_te_acc = te_acc

        if epoch_num % 20 == 0:
            print('%s: at %s at_frac %.1f n_at %d n_mal_sel %d e %d fed_model val loss %.4f val acc %.4f best val_acc %f te_acc %f'%(aggregation, at_type, at_fraction, n_attacker, np.sum(krum_candidate < n_attacker), epoch_num, val_loss, val_acc, best_global_acc,best_global_te_acc))

        epoch_num+=1

==> Initializing global model
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 2 e 0 fed_model val loss 3.9931 val acc 4.6129 best val_acc 4.612850 te_acc 5.369646
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 5 e 20 fed_model val loss 3.8769 val acc 6.8343 best val_acc 7.943781 te_acc 8.700577
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 5 e 40 fed_model val loss 3.8638 val acc 26.1609 best val_acc 26.160935 te_acc 28.995058
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 4 e 60 fed_model val loss 3.5651 val acc 26.7401 best val_acc 29.404345 te_acc 32.822797
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 4 e 80 fed_model val loss 3.0735 val acc 29.2962 best val_acc 31.224259 te_acc 34.686470
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 4 e 100 fed_model val loss 2.7741 val acc 31.4997 best val_acc 33.113674 te_acc 36.686573
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 4 e 120 fed_model val loss 2.5854 val acc 36.8951 best val_acc 36.972302 te_acc 39.731775
bu

bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 6 e 1180 fed_model val loss 1.5827 val acc 60.6338 best val_acc 61.665980 te_acc 63.323723
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 7 e 1200 fed_model val loss 1.7114 val acc 58.4818 best val_acc 61.665980 te_acc 63.323723
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 7 e 1220 fed_model val loss 1.6701 val acc 58.6542 best val_acc 61.665980 te_acc 63.323723
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 6 e 1240 fed_model val loss 1.6507 val acc 59.6710 best val_acc 61.665980 te_acc 63.323723
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 6 e 1260 fed_model val loss 1.6338 val acc 59.8770 best val_acc 61.665980 te_acc 63.323723
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 7 e 1280 fed_model val loss 1.6091 val acc 60.4021 best val_acc 61.665980 te_acc 63.323723
bulyan: at min-max at_frac 20.0 n_at 14 n_mal_sel 8 e 1320 fed_model val loss 1.7948 val acc 57.9592 best val_acc 61.665980 te_acc 63.323723
bulyan: at mi

## Our second AGR-agnostic attack - Min-Sum

In [15]:
def our_attack_score(all_updates, model_re, n_attackers, dev_type='unit_vec'):

    if dev_type == 'unit_vec':
        deviation = model_re / torch.norm(model_re)  # unit vector, dir opp to good dir
    elif dev_type == 'sign':
        deviation = torch.sign(model_re)
    elif dev_type == 'std':
        deviation = torch.std(all_updates, 0)
    
    lamda = torch.Tensor([50.0]).float().cuda()
    # print(lamda)
    threshold_diff = 1e-5
    lamda_fail = lamda
    lamda_succ = 0
    
    distances = []
    for update in all_updates:
        distance = torch.norm((all_updates - update), dim=1) ** 2
        distances = distance[None, :] if not len(distances) else torch.cat((distances, distance[None, :]), 0)
    
    scores = torch.sum(distances, dim=1)
    min_score = torch.min(scores)
    del distances

    while torch.abs(lamda_succ - lamda) > threshold_diff:
        mal_update = (model_re - lamda * deviation)
        distance = torch.norm((all_updates - mal_update), dim=1) ** 2
        score = torch.sum(distance)
        
        if score <= min_score:
            # print('successful lamda is ', lamda)
            lamda_succ = lamda
            lamda = lamda + lamda_fail / 2
        else:
            lamda = lamda - lamda_fail / 2

        lamda_fail = lamda_fail / 2

    mal_update = (model_re - lamda_succ * deviation)
    return mal_update

In [16]:
resume=0
nepochs=1500
gamma=.1
fed_lr=0.001

criterion = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()
batch_size = 100
schedule = [2000]

aggregation = 'bulyan'
at_type = 'min-sum'
chkpt = './' + aggregation
epoch_num = 0

at_fractions = [20]

for at_fraction in at_fractions:

    fed_model = mnist_conv().cuda()
    fed_model.apply(weights_init)
    optimizer_fed = Adam(fed_model.parameters(), lr=fed_lr)

    print('==> Initializing global model')
    epoch_num = 0
    n_attacker = 0
    best_global_acc=0
    best_global_te_acc=0

    while epoch_num <= nepochs:
        user_grads = []

        # The following condition is necessary for Krum/Multi-krum/Bulyan to work
        while n_attacker < 4:
            round_users = np.random.choice(3400, 60)
            n_attacker = np.sum(round_users < (34*at_fraction))

        if n_attacker > 14:
            print ('n_attackers actual %d adjusted 14' % n_attacker)
            n_attacker = 14

        at_idx = []
        attacker_count = 0
        for i in round_users:
            if i < (34*at_fraction) and attacker_count < n_attacker:
                at_idx.append(i)
                attacker_count += 1
                continue

            inputs = user_tr_data_tensors[i]
            targets = user_tr_label_tensors[i]

            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

            outputs = fed_model(inputs)
            loss = criterion(outputs, targets)
            optimizer_fed.zero_grad()
            loss.backward(retain_graph=True)

            param_grad=[]
            for param in fed_model.parameters():
                param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

            user_grads=param_grad[None,:] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]),0)    

        if n_attacker > 0:
            attacker_grads = []
            n_attacker_ = max(1, n_attacker**2//60)
            for i in at_idx:

                inputs = user_tr_data_tensors[i]
                targets = user_tr_label_tensors[i]

                inputs, targets = inputs.cuda(), targets.cuda()
                inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

                outputs = fed_model(inputs)
                loss = criterion(outputs, targets)
                optimizer_fed.zero_grad()
                loss.backward(retain_graph=True)

                param_grad=[]
                for param in fed_model.parameters():
                    param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

                attacker_grads=param_grad[None,:] if len(attacker_grads)==0 else torch.cat((attacker_grads,param_grad[None,:]),0)

            mal_updates = []
            if at_type == 'fang':
                agg_grads = torch.mean(attacker_grads, 0)
                deviation = torch.sign(agg_grads)
                mal_update = get_malicious_updates_fang(attacker_grads, agg_grads, deviation, n_attacker_)
            elif at_type == 'our-agr':
                agg_grads = torch.mean(attacker_grads, 0)
                mal_update = our_attack_mkrum(attacker_grads, agg_grads, n_attacker_, dev_type='sign')
            elif at_type == 'min-max':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_dist(attacker_grads, agg_grads, n_attacker_, dev_type='sign')
            elif at_type == 'min-sum':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_score(attacker_grads, agg_grads, n_attacker_, dev_type='sign')

        if not len(mal_updates):
            mal_updates = torch.stack([mal_update] * n_attacker)
        malicious_grads = torch.cat((mal_updates, user_grads), 0)    
        
        if malicious_grads.shape[0] != 60: 
            print('malicious grads shape ', malicious_grads.shape)
            sys.exit()

        agg_grads, krum_candidate = bulyan(malicious_grads, n_attacker)

        start_idx=0

        if epoch_num in schedule:
            for param_group in optimizer_fed.param_groups:
                param_group['lr'] *= gamma
                print('New learnin rate ', param_group['lr'])

        optimizer_fed.zero_grad()

        model_grads=[]

        for i, param in enumerate(fed_model.parameters()):
            param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
            start_idx=start_idx+len(param.data.view(-1))
            param_=param_.cuda()
            model_grads.append(param_)

        optimizer_fed.step(model_grads)

        val_loss, val_acc = test(val_data_tensor,val_label_tensor,fed_model,criterion,use_cuda)
        te_loss, te_acc = test(te_data_tensor,te_label_tensor, fed_model, criterion, use_cuda)

        is_best = best_global_acc < val_acc

        best_global_acc = max(best_global_acc, val_acc)

        if is_best:
            best_global_te_acc = te_acc

        if epoch_num % 20 == 0:
            print('%s: at %s at_frac %.1f n_at %d n_mal_sel %d e %d fed_model val loss %.4f val acc %.4f best val_acc %f te_acc %f'%(aggregation, at_type, at_fraction, n_attacker, np.sum(krum_candidate < n_attacker), epoch_num, val_loss, val_acc, best_global_acc,best_global_te_acc))

        epoch_num+=1

==> Initializing global model
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 2 e 0 fed_model val loss 3.9542 val acc 5.6631 best val_acc 5.663097 te_acc 6.515136
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 7 e 20 fed_model val loss 3.8183 val acc 11.0945 best val_acc 15.501442 te_acc 16.888900
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 5 e 40 fed_model val loss 3.5834 val acc 24.2509 best val_acc 24.315280 te_acc 26.727245
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 6 e 60 fed_model val loss 3.2242 val acc 24.4646 best val_acc 26.253604 te_acc 29.167525
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 9 e 80 fed_model val loss 3.1330 val acc 27.6462 best val_acc 31.638694 te_acc 35.790774
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 9 e 100 fed_model val loss 3.0015 val acc 31.4637 best val_acc 34.066104 te_acc 37.911862
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 9 e 120 fed_model val loss 2.5655 val acc 37.4691 best val_acc 39.090815 te_acc 43.096170
bulyan

bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 9 e 1180 fed_model val loss 1.2743 val acc 66.3612 best val_acc 69.167010 te_acc 70.855643
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 9 e 1200 fed_model val loss 1.1394 val acc 69.2288 best val_acc 69.864600 te_acc 71.270078
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 9 e 1220 fed_model val loss 1.1540 val acc 67.5582 best val_acc 70.379428 te_acc 71.558381
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 9 e 1240 fed_model val loss 1.1259 val acc 68.9688 best val_acc 70.415465 te_acc 71.591845
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 9 e 1260 fed_model val loss 1.1274 val acc 69.5866 best val_acc 70.876236 te_acc 71.754016
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 9 e 1280 fed_model val loss 1.1709 val acc 69.1902 best val_acc 70.876236 te_acc 71.754016
bulyan: at min-sum at_frac 20.0 n_at 9 n_mal_sel 9 e 1300 fed_model val loss 1.1578 val acc 69.0409 best val_acc 70.876236 te_acc 71.754016
bulyan: at min-sum a